In [ ]:
%load_ext autoreload
%autoreload 2

# add path so we can find files under src/services
import sys
sys.path.append('../src/services')
from src.services import azure_oai
import os
from dotenv import load_dotenv
import pandas as pd
import json
from pathlib import Path

load_dotenv()


In [2]:
import json

# Function to extract the "display" field
def extract_display_field(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
        
        # Navigate to "combinedRecognizedPhrases"
        phrases = data.get('combinedRecognizedPhrases', [])
        
        if not phrases:
            print("No 'combinedRecognizedPhrases' found in the JSON data.")
            return None
        
        # Assuming you want the "display" field from the first item
        first_phrase = phrases[0]
        display_content = first_phrase.get('display')
        
        if display_content:
            return display_content
        else:
            print("'display' field not found in the first 'combinedRecognizedPhrases' item.")
            return None
    
    except FileNotFoundError:
        print(f"The file {file_path} does not exist.")
        return None
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None



In [4]:
#looping through all text file transcripts in data/transcripts
#and calling azure_oai.call_llm() on each one

prompt_file = "./samples/marketing_sentiment_details.txt"
save_transcripts = False
transcripts_folder = "transcripts_whisper_o4"

pure_text = transcripts_folder == "transcripts_whisper_o4"

for filename in os.listdir(f'data/{transcripts_folder}'):
    if pure_text:
        with open(f'data/{transcripts_folder}/{filename}', 'r') as file:
            transcript = file.read()
    else:
        if filename.endswith('.json'):
            transcript = extract_display_field(f'data/{transcripts_folder}/{filename}')
            # save transcript to text file for the streamlit app
            if save_transcripts:
                with open(f'data/{transcripts_folder}/{filename.split(".")[0]}.txt', 'w') as file:
                    file.write(transcript)
    #print(transcript)
    result = azure_oai.call_llm(prompt_file, transcript)
    name = filename.split('.')[0]
    with open(f'data/llm_analysis/{name}.json', 'w') as file:
        file.write(result)
   


### Combine all results in one CSV


In [7]:
def flatten_json(json_data):
    """
    Flatten the nested JSON structure into a single-level dictionary.
    The keys will be in the format 'Parameter X - Metric Name'.
    """
    flat_dict = {}
    for param, metrics in json_data.items():
        for metric, value in metrics.items():
            # Create a unique key for each metric
            flat_key = f"{param} - {metric}"
            flat_dict[flat_key] = value
    return flat_dict


def json_files_to_csv(json_directory, output_csv):
    all_calls = []
    
    # Use pathlib for better path handling
    json_dir = Path(json_directory)
    
    # Check if directory exists
    if not json_dir.exists() or not json_dir.is_dir():
        raise ValueError(f"The directory {json_directory} does not exist or is not a directory.")
    
    # Iterate over all JSON files in the directory
    for json_file in json_dir.glob("*.json"):
        with open(json_file, 'r', encoding='utf-8') as file:
            try:
                data = json.load(file)
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON from file {json_file.name}: {e}")
                continue  # Skip files with invalid JSON
            
            # Flatten the JSON structure
            flat_data = flatten_json(data)
            
            # Add the call identifier (file name without extension)
            call_id = json_file.stem
            flat_data['Call ID'] = call_id
            
            
            # Append the processed data to the list
            all_calls.append(flat_data)
    
    if not all_calls:
        print("No valid JSON files found or all files are invalid.")
        return
    
    # Create a DataFrame
    df = pd.DataFrame(all_calls)
    # Reorder columns to have 'Call ID' as the first column
    columns = ['Call ID'] + [col for col in df.columns if col != 'Call ID']
    df = df[columns]
    
    
    # Optionally, represent the average as a percentage
    # df['Average YES (%)'] = df['Average YES'] * 100
    
    # Save to CSV
    df.to_csv(output_csv, index=False)
    print(f"Combined CSV with summaries saved to {output_csv}")


In [ ]:
json_directory = "./data/llm_analysis/"  
output_csv = "./data/combined_calls.csv"  
    
json_files_to_csv(json_directory, output_csv)

## Merge the results with the ground truth is available

In [9]:
def merge_csv_with_ground_truth(combined_csv_path, ground_truth_excel_path, output_csv_path, drop_unmatched=False):
    """
    Merges the combined CSV with the ground truth data from the Excel file.

    Parameters:
    - combined_csv_path: Path to the combined CSV file generated from JSON files.
    - ground_truth_excel_path: Path to the ground truth Excel file.
    - output_csv_path: Path where the merged CSV will be saved.
    """
    # Check if the combined CSV exists
    if not Path(combined_csv_path).is_file():
        raise FileNotFoundError(f"The combined CSV file {combined_csv_path} does not exist.")
    
    # Check if the Excel file exists
    if not Path(ground_truth_excel_path).is_file():
        raise FileNotFoundError(f"The Excel file {ground_truth_excel_path} does not exist.")
    
    # Read the combined CSV
    try:
        combined_df = pd.read_csv(combined_csv_path)
        print(f"Loaded combined CSV with {len(combined_df)} records.")
    except Exception as e:
        raise Exception(f"Error reading combined CSV: {e}")
    
    # Read the ground truth Excel file
    try:
        ground_truth_df = pd.read_excel(ground_truth_excel_path, sheet_name='Parameters')
        print(f"Loaded ground truth Excel with {len(ground_truth_df)} records.")
    except Exception as e:
        raise Exception(f"Error reading Excel file: {e}")
    
    # Inspect column names
    required_columns = ['Call ID',	'Parameter 1',	'Parameter 2',	'Parameter 3']
    for col in required_columns:
        if col not in ground_truth_df.columns:
            raise ValueError(f"Column '{col}' not found in the 'Parameter' sheet of the Excel file.")
    
    # Filter out rows where 'AUDIO FILE NAME' is not found in ground truth
    ground_truth_df = ground_truth_df[ground_truth_df['Call ID'].notna()]
    ground_truth_df['Call ID'] = ground_truth_df['Call ID'].astype(str)
    # drop column MSDIN
    ground_truth_df = ground_truth_df.drop(columns=['MSDIN'])

    # Convert 'Call ID' columns in both DataFrames to string to ensure consistency
    combined_df['Call ID'] = combined_df['Call ID'].astype(str)
    
    # Check for duplicates in ground truth
    if ground_truth_df['Call ID'].duplicated().any():
        duplicates = ground_truth_df[ground_truth_df['Call ID'].duplicated(keep=False)]
        print(duplicates)
        print("Warning: Duplicate Call IDs found in ground truth data. Using the first occurrence.")
        ground_truth_df = ground_truth_df.drop_duplicates(subset=['Call ID'], keep='first')
    
   
    # Merge the combined CSV with ground truth
    merged_df = pd.merge(combined_df, ground_truth_df, on='Call ID', how='left')
    
    # Check for any Call IDs that didn't find a match
    unmatched = merged_df[merged_df['Parameter 1'].isna()]
    if not unmatched.empty:
        print(f"Warning: {len(unmatched)} Call IDs did not find a matching Parameter X in the ground truth data.")
    
     # If drop_unmatched is True, remove rows that didn't match ground truth
    if drop_unmatched:
        original_count = len(merged_df)
        merged_df = merged_df[merged_df['Parameter 1'].notna()]
        print(f"Dropped {original_count - len(merged_df)} unmatched rows based on ground truth data.")

    try:
        merged_df.to_csv(output_csv_path, index=False)
        print(f"Merged CSV has been saved to {output_csv_path}")
    except Exception as e:
        raise Exception(f"Error saving merged CSV: {e}")


In [ ]:
combined_csv_path = "./data/combined_calls.csv"  # Path to your combined CSV from previous step
ground_truth_excel_path = "./data/sales_call_quality_evaluation.xlsx"  # Path to your ground truth Excel file
output_csv_path = "./data/merged_calls_with_scores.csv"  # Desired output path for the merged CSV
    
# Perform the merge
merge_csv_with_ground_truth(combined_csv_path, ground_truth_excel_path, output_csv_path, True)